In [218]:
import numpy as np
import pandas as pd


In [256]:
values = {'UN Number': ['UN1076', 'UN1075', 'UN1463', 'UN1993', 'NA3077', 'UN1277', 'UN0004', 'UN0005'],
          'Hazard Class': ['3', '3', '6', '8', '9', '4', '6.1', '5.2'],
          'Weight': [300, 800, 450, 1200, 500, 3000, 1001, 1100],
          'Bulk Container?': ['N', 'N', 'Y', 'N', 'Y', 'N', 'N', 'N']}

df = pd.DataFrame.from_dict(values)
df.head(6)

,UN Number,Hazard Class,Weight,Bulk Container?
0,UN1076,3,300,N
1,UN1075,3,800,N
2,UN1463,6,450,Y
3,UN1993,8,1200,N
4,NA3077,9,500,Y
5,UN1277,4,3000,N


In [258]:
#code template for user input: https://www.reddit.com/r/learnpython/comments/rjx5kz/pandas_data_frame_using_user_input/


def placarding(df):

    df2 = df.groupby([
       'Hazard Class'], as_index=False)['Weight'].sum()
    print(df2)
    for row, col in df.iterrows():
        if col['Bulk Container?'] == 'Y':
            print(f"Bulk placard for {col['UN Number']}, {col['Hazard Class']}") 


    if sum(df['Weight']) < 1000:
        print("non-bulk placarding not required")
    for row, col in df2.iterrows():
        if col['Weight'] > 1000 and col['Hazard Class'] != '9':
        	print(f"Placard for Class {col['Hazard Class']}")
        if col['Weight'] < 1000 and col['Hazard Class'] != '9':
                print(f"Use Dangerous placard for Class {col['Hazard Class']} (placarding for Class {col['Hazard Class']} is also acceptable)")
    return df2
	#placeholder for segregation check
    


#def input():
#	input UN number as num
#	input weight as weight
#	input bulk yes/no
#		try:
#	 		where df[UN number] == num, insert weight and bulk
#		except:
#			print "invalid UN number"
#			pass
#	input do you have more to enter?
#		if yes:
#			run input function again
#		if no:
#			pass
#	return placarding()	
			
		

In [260]:
df2 = placarding(df)

  Hazard Class  Weight
0            3    1100
1            4    3000
2          5.2    1100
3            6     450
4          6.1    1001
5            8    1200
6            9     500
Bulk placard for UN1463, 6
Bulk placard for NA3077, 9
Placard for Class 3
Placard for Class 4
Placard for Class 5.2
Use Dangerous placard for Class 6 (placarding for Class 6 is also acceptable)
Placard for Class 6.1
Placard for Class 8


In [262]:
#creating dataframe with all UN numbers and hazard classes

seg_df = pd.read_excel("List of UN numbers and hazard classes.xlsx")


In [264]:
seg_df.head()

,UN Number,Class,Proper Shipping Name
0,UN 0004,1.1D,"Ammonium picrate, dry or wetted with less than..."
1,UN 0005,1.1F,"Cartridges for weapons, with bursting charge"
2,UN 0006,1.1E,"Cartridges for weapons, with bursting charge"
3,UN 0007,1.2F,"Cartridges for weapons, with bursting charge"
4,UN 0009,1.2G,"Ammunition, incendiary with or without burster..."


In [272]:
#build lists of incompatabilities
#we are going to ignore classes 1 and 7 because my employer does not transport those
#we will feed df2 from the function above into this function

#def segregationCheck(df2):
print(df2)
for row, col in df2.iterrows():
    if col['Weight'] > 1000 and col['Hazard Class'] == '2.1':
        if '2.3' in df2['Hazard Class']:
            print("STOP! This shipment may be forbidden or require segregation between classes 2.1 and 2.3.")
        if '6.1' in df2['Hazard Class']:
            print("STOP! This shipment may be forbidden or require segregation between classes 2.1 and 6.1.")

    
    if col['Hazard Class'] == '2.3':
        l1 = ['3', '4.1', '4.2', '4.3', '5.1', '5.2', '8']
        for num in l1:
            if num in df2['Hazard Class']:
                print(f"STOP! This shipment may be forbidden or require segregation between class 2.3 and class {num}")
   
    
    if col['Weight'] > 1000 and col['Hazard Class'] == '3':
        l2 = ['5.1', '6.1']
        for num in l2:
            if num in df2['Hazard Class']:
                print(f"STOP! This shipment may be forbidden or require segregation between class 3 and class {num}")
   
    
    if col['Weight'] > 1000 and col['Hazard Class'] == '4.1': 
        l3 = ['6.1', '8']
        for num in l3:
            if num in df2['Hazard Class']:
                print(f"STOP! This shipment may be forbidden or require segregation between class 4.1 and class {num}")
   
    
    if col['Weight'] > 1000 and col['Hazard Class'] == '4.2':    
        l3 = ['6.1', '8']
        for num in l3:
            if num in df2['Hazard Class']:
                print(f"STOP! This shipment may be forbidden or require segregation between class 4.2 and class {num}")
   
    
    if col['Hazard Class'] == '4.3': 
        l3 = ['6.1', '8']
        for num in l3:
            if num in df2['Hazard Class']:
                print(f"STOP! This shipment may be forbidden or require segregation between class 4.4 and class {num}")
    
    
    if col['Weight'] > 1000 and col['Hazard Class'] == '5.1':  
        l3 = ['6.1', '8']
        for num in l3:
            if num in df2['Hazard Class']:
                print(f"STOP! This shipment may be forbidden or require segregation between class 5.1 and class {num}")
   
    
    if col['Weight'] > 1000 and col['Hazard Class'] == '5.2':   
        l3 = ['6.1', '8']
        for num in l3:
            if num in df2['Hazard Class']:
                print(f"STOP! This shipment may be forbidden or require segregation between class 5.2 and class {num}")
   
    
    if col['Hazard Class'] == '6.1': 
        if '8' in df2['Hazard Class']:
            print("STOP! This shipment may be forbidden or require segregation between class 6.1 and class 8")
    
    

    


  Hazard Class  Weight
0            3    1100
1            4    3000
2          5.2    1100
3            6     450
4          6.1    1001
5            8    1200
6            9     500
